In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import numpy as np
np.bool = np.bool_

In [3]:
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 

In [4]:
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [5]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [6]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executorEnv.PYTHONPATH", "/path/to/your/venv/lib/python3.11/site-packages") \

    .enableHiveSupport()
    .getOrCreate()
)

25/03/18 00:36:00 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
25/03/18 00:36:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e74ec2e9-65c3-4d1a-bf05-a601047b0164;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.0 in central
	found org.tukaani#xz;1.9 in central


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-iceberg_2.12;3.0.0 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
:: resolution report :: resolve 215ms :: artifacts dl 5ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.3 from central in [default]
	com.google.errorprone#error_prone_annotations;2.10.0 from central in [default]
	io.delta#delta-iceberg_2.12;3.0.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.spark#spark-avro_2.12;3.5.0 from central in [default]
	or

In [7]:
trump_tweet_with_sentiment_analysis = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/gold/trump_btc/trump_tweet_with_sentiment_analysis").cache()
btc_price = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/trump_btc/BTC/").cache()


In [8]:
btc_price.orderBy("date").show(5)

+-------------------+----+----+----+-----+------+
|               date|Open|High| Low|Close|Volume|
+-------------------+----+----+----+-----+------+
|2012-01-01 11:01:00|4.58|4.58|4.58| 4.58|   0.0|
|2012-01-01 11:02:00|4.58|4.58|4.58| 4.58|   0.0|
|2012-01-01 11:03:00|4.58|4.58|4.58| 4.58|   0.0|
|2012-01-01 11:04:00|4.58|4.58|4.58| 4.58|   0.0|
|2012-01-01 11:05:00|4.58|4.58|4.58| 4.58|   0.0|
+-------------------+----+----+----+-----+------+
only showing top 5 rows



In [9]:
trump_tweet_with_sentiment_analysis.orderBy("date").show(20)

+-------------------+---------+------------------+---------+--------+--------------------+---------------+
|               date|favorites|                id|isRetweet|retweets|                text|sentiment_score|
+-------------------+---------+------------------+---------+--------+--------------------+---------------+
|2011-07-13 21:34:06|        5| 91228931306426368|    false|      33|More thoughts on ...|        -0.3612|
|2011-07-14 15:36:02|       10| 91501208895881216|    false|      59|"@johnboehner Deb...|        -0.6486|
|2011-07-25 21:41:31|       14| 95579450183385090|    false|      69|The pressure on t...|        -0.5719|
|2011-08-04 22:09:10|       24| 99210286686408704|    false|     182|China demanded th...|        -0.5267|
|2011-08-22 22:33:46|      193|105739460074938368|    false|     299|We don't need ano...|        -0.6908|
|2011-09-06 22:23:34|      105|111172713015545856|    false|     167|@BarackObama tril...|        -0.5563|
|2011-09-09 20:30:23|       23|112231

In [10]:
trump_tweet_with_sentiment_analysis.orderBy("date").show(20)

+-------------------+---------+------------------+---------+--------+--------------------+---------------+
|               date|favorites|                id|isRetweet|retweets|                text|sentiment_score|
+-------------------+---------+------------------+---------+--------+--------------------+---------------+
|2011-07-13 21:34:06|        5| 91228931306426368|    false|      33|More thoughts on ...|        -0.3612|
|2011-07-14 15:36:02|       10| 91501208895881216|    false|      59|"@johnboehner Deb...|        -0.6486|
|2011-07-25 21:41:31|       14| 95579450183385090|    false|      69|The pressure on t...|        -0.5719|
|2011-08-04 22:09:10|       24| 99210286686408704|    false|     182|China demanded th...|        -0.5267|
|2011-08-22 22:33:46|      193|105739460074938368|    false|     299|We don't need ano...|        -0.6908|
|2011-09-06 22:23:34|      105|111172713015545856|    false|     167|@BarackObama tril...|        -0.5563|
|2011-09-09 20:30:23|       23|112231

In [11]:
from pyspark.sql.functions import date_format

#removing the seconds from both DF

trump_tweet_with_sentiment_analysis = trump_tweet_with_sentiment_analysis.withColumn("date", date_format("date", "yyyy-MM-dd HH:mm"))
btc_price = btc_price.withColumn("date", date_format("date", "yyyy-MM-dd HH:mm"))


In [12]:
trump_tweet_with_sentiment_analysis.show(5)
btc_price.show(5)

+----------------+---------+------------------+---------+--------+--------------------+---------------+
|            date|favorites|                id|isRetweet|retweets|                text|sentiment_score|
+----------------+---------+------------------+---------+--------+--------------------+---------------+
|2024-11-04 04:08|        0|113422481640840196|     true|       0|"RT @TeamTrumpPRE...|         0.6597|
|2024-11-04 00:32|    12350|113421631757789522|    false|    3124|"KAMALA BROKE IT,...|          0.976|
|2024-11-03 19:15|        0|113420384878459920|     true|       0|"RT @TeamTrumpPRE...|            0.0|
|2024-11-03 06:25|    17960|113417358038328332|    false|    5386|If Kamala wins, y...|         0.7861|
|2024-11-02 02:02|     8165|113410659723667193|    false|    1919|Patrick Morrisey,...|         0.9824|
+----------------+---------+------------------+---------+--------+--------------------+---------------+
only showing top 5 rows

+----------------+-------+-------+-----

In [13]:
from pyspark.sql.functions import col

#filtering the tweets with sentiment +- 0.5
#filtered_tweets = trump_tweet_with_sentiment_analysis.filter((trump_tweet_with_sentiment_analysis.sentiment_score > 0.5) | (trump_tweet_with_sentiment_analysis.sentiment_score < -0.5))
#filtering because we only got data from 2012
filtered_tweets = trump_tweet_with_sentiment_analysis.filter(col("date") >= "2012-01-01")

filtered_tweets.orderBy('date').show()

+----------------+---------+------------------+---------+--------+--------------------+---------------+
|            date|favorites|                id|isRetweet|retweets|                text|sentiment_score|
+----------------+---------+------------------+---------+--------+--------------------+---------------+
|2012-02-09 20:03|      136|167685018330734592|    false|     185|The Fed is destro...|        -0.5574|
|2012-02-15 22:09|       15|169891053401415680|    false|     106|The recession was...|        -0.8752|
|2012-02-16 21:45|        7|170247520902066176|    false|      55|Gallup poll prove...|        -0.6399|
|2012-02-28 20:54|        3|174583290798415872|    false|      20|@BarackObama's En...|         0.7003|
|2012-03-01 21:02|       15|175310004050591745|    false|     156|"The real unemplo...|            0.0|
|2012-03-19 19:36|       14|181811323947065347|    false|      73|America is mired ...|          -0.34|
|2012-03-30 20:24|        5|185794764657016832|    false|      4

In [14]:
from pyspark.sql import functions as F

# Ensure both DataFrames have the `date` column in TIMESTAMP format
btc_price = btc_price.withColumn("date", F.to_timestamp("date", "yyyy-MM-dd HH:mm"))
filtered_tweets = filtered_tweets.withColumn("date", F.to_timestamp("date", "yyyy-MM-dd HH:mm"))

# Perform the LEFT JOIN on "date"
joined_df = btc_price.join(
    filtered_tweets,
    on="date",
    how="left"
)

# Show results
joined_df.show(10, truncate=False)



+-------------------+-------+-------+-------+-------+-----------+---------+----+---------+--------+----+---------------+
|date               |Open   |High   |Low    |Close  |Volume     |favorites|id  |isRetweet|retweets|text|sentiment_score|
+-------------------+-------+-------+-------+-------+-----------+---------+----+---------+--------+----+---------------+
|2017-07-11 01:18:00|2419.94|2420.0 |2418.95|2418.95|4.07239513 |NULL     |NULL|NULL     |NULL    |NULL|NULL           |
|2017-07-11 01:19:00|2421.99|2422.0 |2419.18|2419.18|15.83349066|NULL     |NULL|NULL     |NULL    |NULL|NULL           |
|2017-07-11 01:20:00|2419.91|2421.96|2419.18|2421.95|2.71926784 |NULL     |NULL|NULL     |NULL    |NULL|NULL           |
|2017-07-11 01:21:00|2421.96|2422.0 |2419.29|2422.0 |42.68201165|NULL     |NULL|NULL     |NULL    |NULL|NULL           |
|2017-07-11 01:22:00|2420.33|2422.98|2414.79|2414.79|11.52859532|NULL     |NULL|NULL     |NULL    |NULL|NULL           |
|2017-07-11 01:23:00|2413.32|241

In [15]:
# Define a 12-hour future time column
from pyspark.sql.functions import col, expr,round

# Create a self-join to find BTC price exactly 24 hours later how it changed over time
joined_df_with_future = joined_df.alias("current").join(
    joined_df.alias("future"),
    expr("future.date = current.date + INTERVAL 24 HOURS"),
    how="left"
).select(
    col("current.text"),
    col("current.date"),
    col("current.sentiment_score"),
    col("current.Close").alias("btc_current_price"),
    col("future.Close").alias("btc_price_24h_later")
)
joined_df_with_future=joined_df_with_future.withColumn(
    "price_change_24h_in_%",
    round(((col("btc_price_24h_later") - col("btc_current_price")) / col("btc_current_price")) * 100
,2))


# Show results
joined_df_with_future.show(10, truncate=False)


+----+-------------------+---------------+-----------------+-------------------+---------------------+
|text|date               |sentiment_score|btc_current_price|btc_price_24h_later|price_change_24h_in_%|
+----+-------------------+---------------+-----------------+-------------------+---------------------+
|NULL|2012-01-01 11:01:00|NULL           |4.58             |5.0                |9.17                 |
|NULL|2012-01-01 11:03:00|NULL           |4.58             |5.0                |9.17                 |
|NULL|2012-01-01 11:08:00|NULL           |4.58             |5.0                |9.17                 |
|NULL|2012-01-01 11:11:00|NULL           |4.58             |5.0                |9.17                 |
|NULL|2017-07-11 01:21:00|NULL           |2422.0           |2326.26            |-3.95                |
|NULL|2017-07-11 01:22:00|NULL           |2414.79          |2324.96            |-3.72                |
|NULL|2017-07-11 01:24:00|NULL           |2409.43          |2325.0       

In [16]:
from pyspark.sql.functions import col, expr, round

# List of rolling windows (in days)
time_windows = [1, 2, 3, 5, 7]  # 1 day = 24h, 2 days = 48h, etc.

# Start with the base DataFrame (1-day window already exists)
joined_df_with_future = joined_df.alias("current")

# Iterate through different time windows and join to calculate future prices
for days in time_windows:
    hours = days * 24  # Convert days to hours
    alias_name = f"future_{days}d"
    
    joined_df_with_future = joined_df_with_future.join(
        joined_df.alias(alias_name),
        expr(f"{alias_name}.date = current.date + INTERVAL {hours} HOURS"),
        how="left"
    ).withColumn(
        f"btc_price_{days}d_later", col(f"{alias_name}.Close")
    ).withColumn(
        f"price_change_{days}d_in_%", 
        round(((col(f"btc_price_{days}d_later") - col("current.Close")) / col("current.Close")) * 100, 2)
    )

# Select relevant columns
final_df = joined_df_with_future.select(
    col("current.text"),
    col("current.date"),
    col("current.sentiment_score"),
    col("current.Close").alias("btc_current_price"),
    *[col(f"btc_price_{days}d_later") for days in time_windows], 
    *[col(f"price_change_{days}d_in_%") for days in time_windows]
)

# Show results


In [17]:
final_df.show(5)


+----+-------------------+---------------+-----------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|text|               date|sentiment_score|btc_current_price|btc_price_1d_later|btc_price_2d_later|btc_price_3d_later|btc_price_5d_later|btc_price_7d_later|price_change_1d_in_%|price_change_2d_in_%|price_change_3d_in_%|price_change_5d_in_%|price_change_7d_in_%|
+----+-------------------+---------------+-----------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|NULL|2012-01-01 11:01:00|           NULL|             4.58|               5.0|               5.0|              4.93|              6.65|               6.8|                9.17|                9.17|                7.64

In [18]:
sentiment_analysis_with_btc_change=final_df.filter(F.col("sentiment_score").isNotNull())

In [19]:
sentiment_analysis_with_btc_change=sentiment_analysis_with_btc_change.cache()

In [20]:
sentiment_analysis_with_btc_change.count()

787

In [21]:
(sentiment_analysis_with_btc_change.write 
                    .format("delta")
                    .mode("overwrite")
                    .option("path",f"{hdfs_lakehouse_base_path}/gold/trump_btc/sentiment_analysis_with_btc_change")
                    .saveAsTable("trump_btc.sentiment_analysis_with_btc_change")
)
